### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 去標點

In [1]:
import json
import nltk
import numpy as np

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [1]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace("，","")
    case_content = case_content.replace("。","")
    case_content = case_content.replace(")","")
    case_content = case_content.replace("(","")
    case_content = case_content.replace("、","")
    case_content = case_content.replace("g","")
    case_content = ''.join(i for i in case_content if not i.isdigit())
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    print(summary_str)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        #print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    texts = []
    for i,w in enumerate(case_content):
        label = getLabelWithWordIndex(i,ne_index_list)
        texts.append((w,label))
        
    docs.append(texts)

print(t)
print(c)

"{""main"": ""寒热不适，发热又需披衣，汗透衣服""# ""pulse"": ""脉虚大而迟""# ""recipe"": {""raw"": """"# ""memo"": """"# ""title"": ""桂枝汤""}# ""second"": """"# ""tongue"": ""苔薄黄""# ""chinese"": ""发热""# ""therapy"": """"# ""western"": """"# ""dialectical"": """"}"


NameError: name 'json' is not defined

In [5]:
print(len(docs))

2107


In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

print(data[0])

[('李', 'JJ', 'O'), ('某', 'NNP', 'O'), ('女', 'NNP', 'O'), ('岁', 'NNP', 'O'), ('每', 'NNP', 'O'), ('于', 'NNP', 'O'), ('夏', 'NNP', 'O'), ('日', 'NNP', 'O'), ('洗', 'NNP', 'O'), ('澡', 'NNP', 'O'), ('后', 'NNP', 'O'), ('即', 'NNP', 'O'), ('寒', 'NNP', 'B-S'), ('热', 'NNP', 'I-S'), ('不', 'NNP', 'I-S'), ('适', 'NNP', 'E-S'), ('自', 'NNP', 'O'), ('感', 'NNP', 'O'), ('发', 'NNP', 'O'), ('热', 'NNP', 'O'), ('而', 'NNP', 'O'), ('又', 'NNP', 'O'), ('需', 'NNP', 'O'), ('披', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('裹', 'NNP', 'O'), ('护', 'NNP', 'O'), ('并', 'NNP', 'O'), ('于', 'NNP', 'O'), ('不', 'NNP', 'O'), ('知', 'NNP', 'O'), ('不', 'NNP', 'O'), ('觉', 'NNP', 'O'), ('中', 'NNP', 'O'), ('汗', 'NNP', 'B-S'), ('透', 'NNP', 'I-S'), ('衣', 'NNP', 'I-S'), ('服', 'NNP', 'E-S'), ('急', 'NNP', 'O'), ('又', 'NNP', 'O'), ('换', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('如', 'NNP', 'O'), ('此', 'NNP', 'O'), ('三', 'NNP', 'O'), ('四', 'NNP', 'O'), ('次', 'NNP', 'O'), ('方', 'NNP', 'O'), ('渐', 'NNP', 'O'), ('趋', 'NNP', 'O'), ('平', 'NNP', 'O'), ('复', 'NNP', 'O

In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train[0][4]))
print(X_train[0][5])

18
['bias', 'word.lower=代', 'word[-3:]=代', 'word[-2:]=代', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNP', '-1:word.lower=年', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NNP', '+1:word.lower=我', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP']


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30365
Seconds required: 2.145

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 179861.035575
Feature norm: 1.000000
Error norm: 39658.272625
Active features: 30248
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.828

***** Iteration #2 *****
Loss: 179349.120526
Feature norm: 0.997889
Error norm: 39459.077073
Active features: 29047
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.438

***** Iteration #3 *****
Loss: 145434.225796
Feature norm: 4.237426
Error norm: 147486.707431
Active features: 16326
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #40 *****
Loss: 34575.973572
Feature norm: 32.531511
Error norm: 11349.338169
Active features: 26270
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.427

***** Iteration #41 *****
Loss: 33033.225779
Feature norm: 36.548755
Error norm: 25449.191630
Active features: 24401
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.419

***** Iteration #42 *****
Loss: 32130.713672
Feature norm: 37.718445
Error norm: 16836.005485
Active features: 23347
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.419

***** Iteration #43 *****
Loss: 31589.433547
Feature norm: 39.071768
Error norm: 29645.824993
Active features: 23487
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.426

***** Iteration #44 *****
Loss: 30961.812557
Feature norm: 39.998964
Error norm: 5607.419535
Active features: 23194
Line search trials: 1
Line search step: 1.00

***** Iteration #87 *****
Loss: 23517.754063
Feature norm: 91.049153
Error norm: 11725.579901
Active features: 21668
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.431

***** Iteration #88 *****
Loss: 23474.564126
Feature norm: 92.096868
Error norm: 11258.398808
Active features: 21633
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.423

***** Iteration #89 *****
Loss: 23433.552287
Feature norm: 92.388310
Error norm: 5578.342041
Active features: 21673
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.424

***** Iteration #90 *****
Loss: 23403.423022
Feature norm: 92.727306
Error norm: 2820.024326
Active features: 21652
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.422

***** Iteration #91 *****
Loss: 23358.413640
Feature norm: 93.217630
Error norm: 3768.339182
Active features: 21599
Line search trials: 1
Line search step: 1.0000

***** Iteration #132 *****
Loss: 22636.334961
Feature norm: 105.164363
Error norm: 6121.165926
Active features: 20594
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.426

***** Iteration #133 *****
Loss: 22625.872474
Feature norm: 105.666598
Error norm: 4414.693035
Active features: 20587
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.421

***** Iteration #134 *****
Loss: 22617.724350
Feature norm: 105.800988
Error norm: 2201.382748
Active features: 20633
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.426

***** Iteration #135 *****
Loss: 22611.157471
Feature norm: 105.975531
Error norm: 1102.781475
Active features: 20630
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.420

***** Iteration #136 *****
Loss: 22599.195491
Feature norm: 106.279640
Error norm: 1151.543114
Active features: 20580
Line search trials: 1
Line search step

***** Iteration #173 *****
Loss: 22413.404692
Feature norm: 113.394151
Error norm: 3613.391076
Active features: 20326
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.425

***** Iteration #174 *****
Loss: 22409.893638
Feature norm: 113.466031
Error norm: 1762.709714
Active features: 20316
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.419

***** Iteration #175 *****
Loss: 22407.069851
Feature norm: 113.567189
Error norm: 2118.130432
Active features: 20338
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.422

***** Iteration #176 *****
Loss: 22404.196420
Feature norm: 113.640460
Error norm: 1660.977406
Active features: 20318
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.419

***** Iteration #177 *****
Loss: 22402.940735
Feature norm: 113.799703
Error norm: 4132.643018
Active features: 20310
Line search trials: 1
Line search step

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

高 (O)
某 (O)
男 (O)
岁 (O)
初 (O)
诊 (O)
年 (O)
月 (O)
日 (O)
主 (O)
诉 (O)
腰 (B-S)
酸 (I-S)
浮 (I-S)
肿 (E-S)
蛋 (B-S)
白 (I-S)
尿 (E-S)
个 (O)
月 (O)
病 (O)
史 (O)
患 (O)
者 (O)
于 (O)
年 (O)
月 (O)
起 (O)
腰 (O)
痠 (O)
伴 (O)
夜 (O)
尿 (O)
增 (O)
多 (O)
至 (O)
月 (O)
份 (O)
尿 (O)
频 (O)
尿 (O)
急 (O)
明 (O)
显 (O)
当 (O)
地 (O)
医 (O)
院 (O)
查 (O)
尿 (O)
常 (O)
规 (O)
： (O)
蛋 (O)
白 (O)
（ (O)
+ (O)
） (O)
红 (O)
细 (O)
胞 (O)
（ (O)
+ (O)
+ (O)
） (O)
曾 (O)
给 (O)
予 (O)
吡 (O)
哌 (O)
酸 (O)
先 (O)
锋 (O)
i (O)
v (O)
号 (O)
等 (O)
治 (O)
疗 (O)
无 (O)
效 (O)
转 (O)
市 (O)
某 (O)
医 (O)
院 (O)
拟 (O)
诊 (O)
“ (O)
肾 (O)
小 (O)
球 (O)
肾 (O)
炎 (O)
” (O)
住 (O)
院 (O)
一 (O)
个 (O)
月 (O)
曾 (O)
作 (O)
肾 (O)
穿 (O)
刺 (O)
示 (O)
： (O)
肾 (O)
小 (O)
球 (O)
局 (O)
灶 (O)
性 (O)
硬 (O)
化 (O)
因 (O)
症 (O)
情 (O)
控 (O)
制 (O)
不 (O)
满 (O)
意 (O)
而 (O)
来 (O)
中 (O)
医 (O)
门 (O)
诊 (O)
目 (O)
前 (O)
仍 (O)
感 (O)
腰 (B-S)
部 (I-S)
痠 (I-S)
痛 (E-S)
下 (B-S)
肢 (I-S)
浮 (I-S)
肿 (E-S)
夜 (B-S)
寐 (I-S)
梦 (I-S)
多 (E-S)
舌 (O)
脉 (O)
舌 (O)
苔 (O)
薄 (O)
黄 (O)
少 (O)
润 (O)
脉 (O)
细 (O)
检 (O)
查 (O)
血 (O)
压 (O)
/ (O)
m 

In [12]:
from sklearn.metrics import accuracy_score

# Convert the sequences of tags into a 1-dimensional array
#predictions = np.array([labels[tag] for row in y_pred for tag in row])
#truths = np.array([labels[tag] for row in y_test for tag in row])

print(y_pred)

#print(accuracy_score(y_test, y_pred))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [13]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       0.97      1.00      0.98    254738
        B-S       0.45      0.12      0.19      2386
        I-S       0.40      0.10      0.16      3599
        E-S       0.47      0.13      0.20      2380

avg / total       0.96      0.97      0.96    263103

